In [10]:
# LIBRERIAS NECESARIAS
import pandas as pd
import requests
from bs4 import BeautifulSoup
import simple_icd_10 as icd

In [48]:
# OBTENER CODIGOS
codigos = icd.get_all_codes()

# FILTRAR CAPITULOS Y BLOQUES
codigos_filtrados = []
for codigo in codigos:
    if not icd.is_chapter_or_block(codigo):
        codigos_filtrados.append(codigo)
# DESCRIPCIONES
descripciones = [icd.get_description(codigo) for codigo in codigos_filtrados]

In [50]:
# SCRAPPING
from tqdm import tqdm
url_base = "https://en.wikipedia.org/w/index.php?search="
sintomas = []
for descripcion in tqdm(descripciones):
    req = requests.get(url_base + descripcion)
    soup = BeautifulSoup(req.content)
    res = soup.find(title="Signs and symptoms")
    try:
        sintomas.append(res.parent.parent.find("td").text)
    except: 
        sintomas.append("Sintomas no encontrados")

100%|██████████████████████████████████████████████████████████████████████████| 12246/12246 [2:40:42<00:00,  1.27it/s]


In [51]:
data = pd.DataFrame({"codigo": codigos_filtrados, "sintomas": sintomas})

In [156]:
data.to_csv("./icd_sintomas.csv")